In [10]:
import pandas as pd
import ast

In [11]:
df= pd.read_csv('word_piece_based.csv')
df.drop(labels= 'Unnamed: 0', axis=1, inplace= True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company_name         2996 non-null   object
 1   description          2996 non-null   object
 2   website              2996 non-null   object
 3   funding              2996 non-null   object
 4   industries           2996 non-null   object
 5   num_employees        2996 non-null   object
 6   headquarters         2996 non-null   object
 7   founded_date         2996 non-null   object
 8   industry_groups      2996 non-null   object
 9   full_description     2996 non-null   object
 10  filter               2996 non-null   object
 11  cleaned_description  2996 non-null   object
 12  tokenized text       2996 non-null   object
dtypes: object(13)
memory usage: 304.4+ KB


In [13]:
df.drop_duplicates(subset='company_name', inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2787 entries, 0 to 2995
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company_name         2787 non-null   object
 1   description          2787 non-null   object
 2   website              2787 non-null   object
 3   funding              2787 non-null   object
 4   industries           2787 non-null   object
 5   num_employees        2787 non-null   object
 6   headquarters         2787 non-null   object
 7   founded_date         2787 non-null   object
 8   industry_groups      2787 non-null   object
 9   full_description     2787 non-null   object
 10  filter               2787 non-null   object
 11  cleaned_description  2787 non-null   object
 12  tokenized text       2787 non-null   object
dtypes: object(13)
memory usage: 304.8+ KB


In [15]:
df.head()

,company_name,description,website,funding,industries,num_employees,headquarters,founded_date,industry_groups,full_description,filter,cleaned_description,tokenized text
0,1 Accord Technologies,1 accord technology providing professional sup...,https://www.ashton-group.com/,Na,"Consulting, Cyber Security, Hardware, IT Mana...",01 to 10,"Eatonton, Georgia, United States",2018,"Hardware, Information Technology, Privacy and...",—,Software-2018,1 accord technology providing professional sup...,"['1', 'accord', 'technology', 'providing', 'pr..."
1,1005 Ventures,venture capital,http://www.1005ventures.com,Na,"Information Technology, Internet, Venture Cap...",01 to 10,"San Francisco, California, United States",2018,"Financial Services, Information Technology, I...",1005 ventures is set up to invest in start-ups...,Software-2018,venture capital,"['venture', 'capital']"
2,"1533 Systems, Inc.",live data platform ingest process respond oper...,http://1533.io,Na,"Analytics, Information Technology, Software, ...",01 to 10,"Campbell, California, United States",2018,"Content and Publishing, Data and Analytics, I...",1533 Systems is building the next generation d...,Software-2018,live data platform ingest process respond oper...,"['live', 'data', 'platform', 'ing', '##est', '..."
3,17 TeraWatts,17 terawatts clean energy software company,https://www.17terawatts.com/,Na,"Clean Energy, Energy, Information Technology,...",01 to 10,"Austin, Texas, United States",2018-01-01 00:00:00,"Energy, Information Technology, Software, Sus...",—,Software-2018,17 terawatts clean energy software company,"['17', 'te', '##raw', '##att', '##s', 'clean',..."
4,3DIQ,3diq make beyond easy recruiter edit format ge...,https://www.3diq.com,Na,"Recruiting, Software",01 to 10,"Knoxville, Tennessee, United States",2018,"Professional Services, Software",3DIQ is a recruiter’s secret weapon. We make i...,Software-2018,3diq make beyond easy recruiter edit format ge...,"['3', '##di', '##q', 'make', 'beyond', 'easy',..."


In [16]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [17]:
import json
import logging
from re import sub
from multiprocessing import cpu_count

import numpy as np

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
#from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [18]:
if 'glove' not in locals():  # only load if not already in memory
    glove = api.load("glove-wiki-gigaword-50")
    
similarity_index = WordEmbeddingSimilarityIndex(glove)

[==================================================] 100.0% 66.0/66.0MB downloaded


In [19]:
ind=1000 #the query

In [20]:
query= df.loc[ind]['cleaned_description']
query_description= df.loc[ind]['description'] 
df.drop(labels= ind, inplace= True, axis=0) #dropping that query 
df.head()

,company_name,description,website,funding,industries,num_employees,headquarters,founded_date,industry_groups,full_description,filter,cleaned_description,tokenized text
0,1 Accord Technologies,1 accord technology providing professional sup...,https://www.ashton-group.com/,Na,"Consulting, Cyber Security, Hardware, IT Mana...",01 to 10,"Eatonton, Georgia, United States",2018,"Hardware, Information Technology, Privacy and...",—,Software-2018,1 accord technology providing professional sup...,"['1', 'accord', 'technology', 'providing', 'pr..."
1,1005 Ventures,venture capital,http://www.1005ventures.com,Na,"Information Technology, Internet, Venture Cap...",01 to 10,"San Francisco, California, United States",2018,"Financial Services, Information Technology, I...",1005 ventures is set up to invest in start-ups...,Software-2018,venture capital,"['venture', 'capital']"
2,"1533 Systems, Inc.",live data platform ingest process respond oper...,http://1533.io,Na,"Analytics, Information Technology, Software, ...",01 to 10,"Campbell, California, United States",2018,"Content and Publishing, Data and Analytics, I...",1533 Systems is building the next generation d...,Software-2018,live data platform ingest process respond oper...,"['live', 'data', 'platform', 'ing', '##est', '..."
3,17 TeraWatts,17 terawatts clean energy software company,https://www.17terawatts.com/,Na,"Clean Energy, Energy, Information Technology,...",01 to 10,"Austin, Texas, United States",2018-01-01 00:00:00,"Energy, Information Technology, Software, Sus...",—,Software-2018,17 terawatts clean energy software company,"['17', 'te', '##raw', '##att', '##s', 'clean',..."
4,3DIQ,3diq make beyond easy recruiter edit format ge...,https://www.3diq.com,Na,"Recruiting, Software",01 to 10,"Knoxville, Tennessee, United States",2018,"Professional Services, Software",3DIQ is a recruiter’s secret weapon. We make i...,Software-2018,3diq make beyond easy recruiter edit format ge...,"['3', '##di', '##q', 'make', 'beyond', 'easy',..."


In [21]:
documents=[]
corpus=[]
for i in range(len(df)):
  #corpus.append(ast.literal_eval(df.iloc[i]['tokenized text']))
  documents.append((df.iloc[i]['cleaned_description'])) #create a list of cleaned discription documents
    
  

In [22]:
documents[0], query

('1 accord technology providing professional support business strive give client enterpriselevel service',
 'postalytics selfserve direct mail automation tool look act feel like digital channel')

In [23]:
def preprocess(doc):
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf"))] #tokenization

In [24]:
corpus = [preprocess(document) for document in documents]
query_token = preprocess(query)

In [25]:
dictionary = Dictionary(corpus+[query_token])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix. 
# The nonzero_limit enforces sparsity by limiting the number of non-zero terms in each column. 
# For my application, I got best results by removing the default value of 100
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)  # , nonzero_limit=None)

100%|██████████| 6077/6077 [01:10<00:00, 85.72it/s]


In [26]:
query_tf = tfidf[dictionary.doc2bow(query_token)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [27]:
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
print("query=", query_description)
for idx in sorted_indexes[:50]:
    if doc_similarity_scores[idx] > 0.500:
      print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {df.iloc[idx]["description"]}')

query= postalytics selfserve direct mail automation tool look act feel like digital channel
2385 	 0.614 	 little otter childrens mental health company developed app focus mental health child family
1184 	 0.614 	 little acorn shave retailing company
832 	 0.600 	 scoremedia platform measure score thing done life
1185 	 0.586 	 little lion provides family activity nutrition yoga kickboxing art wellness via online interactive platform
1107 	 0.581 	 ondemand moving application help people find storage facility sell unwanted good move item demand
822 	 0.552 	 seedership powering small firm good community digital platform help keep act kindness visible
2143 	 0.542 	 kuri edited know eat inc first sustainable cooking app available io
430 	 0.536 	 hovercast entertainment company specializes field internet thing marketing social medium
1408 	 0.536 	 grabit ondemand pickup delivery service order anything anywhere anytime
436 	 0.532 	 zunta technology provides service like cloud disaster 